# Trabajo Pracatico 4 - Redes Neuronales Convolucionales

Vamos a realizar una clasificacion binaria de imagnes que pueden ser de gatos o perros, para ello vamos a utilizar un dataset de Kaggle llamado "cats-vs-dogs" que contiene 23.409 imágenes de gatos y perros. El objetivo es entrenar un modelo de clasificación binaria que pueda distinguir entre imágenes de gatos y perros.

Se van a proponer los siguientes modelos:

- **Modelo 1:** Red convolucional simple (SimpleCNN) creada desde cero.
- **Modelo 2:** Red convolucional basada en ResNet18.
- **Modelo 4:** Red convolucional avanzacda (AdvancedCNN) creada desde cero.
- **Modelo 3:** Red convolucional basada en ResNet18 con cambios en hiperparametros.
- **Modelo 5:** Red convolucional basada en Inception de Google con cambios en hiperparametros.

Asignamos el dataset a la variable **dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cats_vs_dogs")

Creamos un *DataFrame* llamado **mydataset**, el cual almacenará el path de cada imágen junto a su etiqueta (perro o gato). Además creamos un directorio llamado dataset y almacenamos allí las imágenes.


In [ ]:
import pandas as pd
import os

main_dir = './dataset'
os.makedirs(main_dir, exist_ok=True)

mydataset = pd.DataFrame(columns=['image_path', 'label'])

for i in range(len(dataset['train'])):
    img_path = f"{main_dir}/img_{i}.jpeg"

    if not os.path.exists(img_path):
        dataset['train'][i]['image'].save(img_path)

    mydataset.at[i, 'image_path'] = img_path
    mydataset.at[i, 'label'] = dataset['train'][i]['labels']

mydataset.head()

Creamos un diccionario para almacenar los parámetros que usaremos.

In [ ]:
exp_config = dict()

Definimos la semilla para que al divir el dataset en train, test y val, sea siempre la misma división de datos. Además, especificamos la proporción de datos que serán para testeo y para validación.

In [ ]:
seed = 42
test_size = 0.15
val_size = 0.20

exp_config['seed'] = seed
exp_config['test_size'] = test_size
exp_config['val_size'] = val_size

Dividimos el dataset en *train*, *test*, *val*.

**Aclaración:** los datos de validación surgen de una parte de los datos de testeo.

In [ ]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(mydataset, test_size=test_size, stratify=mydataset['label'], random_state=seed)

train_df, val_df = train_test_split(train_val_df, test_size=val_size, stratify=train_val_df['label'], random_state=seed)

Añadimos parámetros de configuración al diccionario.

In [ ]:
exp_config['train_n_cats'] = train_df['label'].value_counts()[0]
exp_config['train_n_dogs'] = train_df['label'].value_counts()[1]
exp_config['val_n_cats'] = val_df['label'].value_counts()[0]
exp_config['val_n_dogs'] = val_df['label'].value_counts()[1]
exp_config['test_n_cats'] = test_df['label'].value_counts()[0]
exp_config['test_n_dogs'] = test_df['label'].value_counts()[1]

La clase **CatsDogsDataset** es una implementación personalizada de una clase llamda *Dataset* de PyTorch que permite cargar y transformar las imágenes del dataset.

**Explicación**
1. Constructor (\_\_init\_\_):  
- img_path_list: Lista de rutas de las imágenes.
- lab_list: Lista de etiquetas correspondientes a las imágenes (0 para gatos, 1 para perros).
- transform: Transformaciones opcionales que se aplicarán a las imágenes (por ejemplo, redimensionar, normalizar).
2. Método \_\_len\_\_:  
- Devuelve la cantidad de imágenes en el conjunto de datos.
3. Método \_\_getitem\_\_:
- idx: Índice de la imagen y etiqueta que se desea obtener.
- img_path: Obtiene la ruta de la imagen en el índice idx.
- image: Abre la imagen y la convierte a formato RGB.
- label: Obtiene la etiqueta correspondiente a la imagen y la convierte a un tensor de PyTorch.
- Si se especificaron transformaciones, se aplican a la imagen.
- Devuelve la imagen transformada y su etiqueta correspondiente.

In [ ]:
from PIL import Image
import torch
from torch.utils.data import Dataset

class CatsDogsDataset(Dataset):
    def __init__(self, img_path_list, lab_list, transform=None):
        self.transform = transform
        self.images = img_path_list
        self.labels = lab_list

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")

        label = self.labels[idx]
        label = torch.Tensor([label])

        if self.transform:
            image = self.transform(image)

        return image, label

Definimos la resolución de las imágenes que serán procesadas.

In [ ]:
input_size = (224,224)
exp_config['input_size'] = input_size

Como las imágenes son a color en formato RGB, definiremos 3 canales

In [ ]:
n_channels = 3
exp_config['n_channels'] = n_channels

Creamos el *transform* que será usado, el cual redimensiona las imágenes a la resolución dada.

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
])

Creamos los datasets de train, test y val.

In [ ]:
train_dataset = CatsDogsDataset(train_df['image_path'].tolist(), train_df['label'].tolist(), transform)
test_dataset = CatsDogsDataset(test_df['image_path'].tolist(), test_df['label'].tolist(), transform)
val_dataset = CatsDogsDataset(val_df['image_path'].tolist(), val_df['label'].tolist(), transform)

Creamos los *DataLoaders* de train, test y val, y definimos el tamaño de lote.

**Aclaración:** el batch size de test es 1,los datos no serán mezclados por cada época y no se eliminarán datos para alcanzar el tamaño de lote establecido.

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64
exp_config['batch_size'] = batch_size

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)

## WandB

In [ ]:
import wandb

wandb.login(key="d567fa512c6502cc7986d8c90fd37c4f0969de0d")

Definimos el dispositivo donde se realizará el entrenamiento (CPU o GPU).

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

NameError: name 'torch' is not defined

# Uso de los CNNs

In [ ]:
from tools.models import SimpleCNN, ResNet18, AdvancedCNN, InceptionCNN, DenseNet_121CNN
from tools.utils import train_and_validate, test, classify, calculate_metrics

base_path = './checkpoints'

os.makedirs(base_path, exist_ok=True)

## SimpleCNN

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim
from torch import nn

exp_config_SimpleCNN = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_SimpleCNN")
wandb.config.update(exp_config_SimpleCNN)

model = SimpleCNN().to(device)
exp_config_SimpleCNN['model'] = 'SimpleCNN'

criterion = nn.BCELoss()
exp_config_SimpleCNN['model'] = 'BCELoss'

lr = 0.001
exp_config_SimpleCNN['learning_rate'] = lr

optimizer = optim.Adam(model.parameters(), lr=lr)
exp_config_SimpleCNN['optimizador'] = 'Adam'

### Ajuste del modelo

Aqui definimos la cantidad de epocas y el criterio que va a tener en cuenta para detener el entrenamiento en caso de no ver mejoras.

In [ ]:
num_epochs = 15
early_stopping_patience = 5
epochs_without_improvement = 0

exp_config_SimpleCNN['num_epochs'] = num_epochs
exp_config_SimpleCNN['early_stopping_patience'] = early_stopping_patience

checkpoint_path = base_path + '/best_model.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

## Testeo

Cargamos los parametros del modelo desde el checkpoint.

In [ ]:
model = SimpleCNN().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

Hacemos predicciones en el conjunto de test.

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

Pasamos las predicciones a tensores y clasificamos en base a un umbral.

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

### Métricas

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

roc_data = [[x, y] for (x, y) in zip(fpr, tpr)]
table = wandb.Table(data=roc_data, columns=["FPR", "TPR"])
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_specificity": specificity,
    "test_confusion_matrix": wandb.plot.confusion_matrix(y_true=y_true.flatten().tolist(), preds=y_pred.flatten().tolist(), class_names=["Clase 0", "Clase 1"]),
    "ROC Curve": wandb.plot.line(table, "FPR", "TPR", title="ROC Curve"),
    "test_roc_auc": roc_auc,
})

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import json
import os

dicts_path = './dicts'

os.makedirs(dicts_path, exist_ok=True)

with open(dicts_path + '/exp_config_SimpleCNN.json', 'w') as json_file:
    json.dump(exp_config_SimpleCNN, json_file, indent=4)
    
wandb.finish()

exp_config_SimpleCNN

## ResNet18

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim

exp_config_ResNet18 = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_ResNet18CNN")
wandb.config.update(exp_config_ResNet18)

model = ResNet18().to(device)
exp_config_ResNet18['model'] = 'ResNet18'

criterion = nn.BCELoss()
exp_config_ResNet18['model'] = 'BCELoss'

lr = 0.001
exp_config_ResNet18['learning_rate'] = lr

optimizer = optim.Adam(model.parameters(), lr=lr)
exp_config_ResNet18['optimizador'] = 'Adam'

model

### Ajuste

In [ ]:
num_epochs = 15
early_stopping_patience = 5

exp_config_ResNet18['num_epochs'] = num_epochs
exp_config_ResNet18['early_stopping_patience'] = early_stopping_patience

checkpoint_path = base_path + '/best_model_ResNet18.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

### Test

In [ ]:
model = ResNet18().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

roc_data = [[x, y] for (x, y) in zip(fpr, tpr)]
table = wandb.Table(data=roc_data, columns=["FPR", "TPR"])
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_specificity": specificity,
    "test_confusion_matrix": wandb.plot.confusion_matrix(y_true=y_true.flatten().tolist(), preds=y_pred.flatten().tolist(), class_names=["Clase 0", "Clase 1"]),
    "ROC Curve": wandb.plot.line(table, "FPR", "TPR", title="ROC Curve"),
    "test_roc_auc": roc_auc,
})

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import json
import os

dicts_path = './dicts'

os.makedirs(dicts_path, exist_ok=True)

with open(dicts_path + '/exp_config_ResNet18.json', 'w') as json_file:
    json.dump(exp_config_ResNet18, json_file, indent=4)
    
wandb.finish()

exp_config_ResNet18

## ResNet 18 Modificado

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim

exp_config_ResNet18Modificado = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_ResNet18ModificadoCNN")
wandb.config.update(exp_config_ResNet18Modificado)

model = ResNet18().to(device)
exp_config_ResNet18Modificado['model'] = 'ResNet18Modificado'

criterion = nn.BCELoss()
exp_config_ResNet18Modificado['model'] = 'BCELoss'

lr = 0.001
exp_config_ResNet18Modificado['learning_rate'] = lr

# Se usara otro optimizador

weight_decay = 0.01
exp_config_ResNet18Modificado['weight_decay'] = weight_decay

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

exp_config_ResNet18Modificado['optimizador'] = 'AdamW'

model

### Ajuste

In [ ]:
num_epochs = 15
early_stopping_patience = 5

exp_config_ResNet18Modificado['num_epochs'] = num_epochs
exp_config_ResNet18Modificado['early_stopping_patience'] = early_stopping_patience

checkpoint_path = base_path + '/best_model_ResNet18.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

### Test

In [ ]:
model = ResNet18().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

roc_data = [[x, y] for (x, y) in zip(fpr, tpr)]
table = wandb.Table(data=roc_data, columns=["FPR", "TPR"])
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_specificity": specificity,
    "test_confusion_matrix": wandb.plot.confusion_matrix(y_true=y_true.flatten().tolist(), preds=y_pred.flatten().tolist(), class_names=["Clase 0", "Clase 1"]),
    "ROC Curve": wandb.plot.line(table, "FPR", "TPR", title="ROC Curve"),
    "test_roc_auc": roc_auc,
})

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import json
import os

dicts_path = './dicts'

os.makedirs(dicts_path, exist_ok=True)

with open(dicts_path + '/exp_config_ResNet18Modificado.json', 'w') as json_file:
    json.dump(exp_config_ResNet18Modificado, json_file, indent=4)
    
wandb.finish()

exp_config_ResNet18Modificado

## AdvancedCNN

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim

exp_config_AdvancedCNN = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_AdvancedCNN")
wandb.config.update(exp_config_AdvancedCNN)

model = AdvancedCNN().to(device)
exp_config_AdvancedCNN['model'] = 'AdvancedCNN'

criterion = nn.BCELoss()
exp_config_AdvancedCNN['model'] = 'BCELoss'

lr = 0.001
exp_config_AdvancedCNN['learning_rate'] = lr

optimizer = optim.Adam(model.parameters(), lr=lr)
exp_config_AdvancedCNN['optimizador'] = 'Adam'

model

### Ajuste

In [ ]:
num_epochs = 15
early_stopping_patience = 5

exp_config_AdvancedCNN['num_epochs'] = num_epochs
exp_config_AdvancedCNN['early_stopping_patience'] = early_stopping_patience

checkpoint_path = base_path + '/best_model_AdvancedCNN.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

### Test

In [ ]:
model = AdvancedCNN().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

Hacemos predicciones en el conjunto de test.

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

roc_data = [[x, y] for (x, y) in zip(fpr, tpr)]
table = wandb.Table(data=roc_data, columns=["FPR", "TPR"])
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_specificity": specificity,
    "test_confusion_matrix": wandb.plot.confusion_matrix(y_true=y_true.flatten().tolist(), preds=y_pred.flatten().tolist(), class_names=["Clase 0", "Clase 1"]),
    "ROC Curve": wandb.plot.line(table, "FPR", "TPR", title="ROC Curve"),
    "test_roc_auc": roc_auc,
})

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import json
import os

dicts_path = './dicts'

os.makedirs(dicts_path, exist_ok=True)

with open(dicts_path + '/exp_config_AdvancedCNN.json', 'w') as json_file:
    json.dump(exp_config_AdvancedCNN, json_file, indent=4)
    
wandb.finish()

exp_config_AdvancedCNN

## DenseNet_121CNN

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim

exp_config_DenseNet_121C = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_DenseNet_121CNN")
wandb.config.update(exp_config_DenseNet_121C)

model = InceptionCNN().to(device)
exp_config_DenseNet_121C['model'] = 'DenseNet_121CNN'

criterion = nn.BCELoss()
exp_config_DenseNet_121C['model'] = 'BCELoss'

lr = 0.001
exp_config_DenseNet_121C['learning_rate'] = lr

optimizer = optim.Adam(model.parameters(), lr=lr)
exp_config_DenseNet_121C['optimizador'] = 'Adam'

model

### Ajuste

In [ ]:
num_epochs = 15
early_stopping_patience = 5

exp_config_DenseNet_121C['num_epochs'] = num_epochs
exp_config_DenseNet_121C['early_stopping_patience'] = early_stopping_patience

checkpoint_path = base_path + '/best_model_DenseNet_121CNN.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

### Test

In [ ]:
model = DenseNet_121CNN().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

roc_data = [[x, y] for (x, y) in zip(fpr, tpr)]
table = wandb.Table(data=roc_data, columns=["FPR", "TPR"])
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_specificity": specificity,
    "test_confusion_matrix": wandb.plot.confusion_matrix(y_true=y_true.flatten().tolist(), preds=y_pred.flatten().tolist(), class_names=["Clase 0", "Clase 1"]),
    "ROC Curve": wandb.plot.line(table, "FPR", "TPR", title="ROC Curve"),
    "test_roc_auc": roc_auc,
})

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
exp_config_DenseNet_121C

## DenseNet_121 Modificado

### Elección de modelo, función de costo y optimizador.

In [ ]:
import torch.optim as optim

exp_config_DenseNet_121Modificado = exp_config.copy()

wandb.init(project="CNN_CatsvsDogs", entity="ar-um", tags=["BERTOLDI_MANCUSO"], name="Bertoldi_Mancuso_InceptionModificadoCNN")
wandb.config.update(exp_config_DenseNet_121Modificado)

model = DenseNet_121CNN().to(device)
exp_config_DenseNet_121Modificado['model'] = 'InceptionModificado'

criterion = nn.BCELoss()
exp_config_DenseNet_121Modificado['model'] = 'BCELoss'

lr = 0.01
exp_config_DenseNet_121Modificado['learning_rate'] = lr

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

exp_config_DenseNet_121Modificado['optimizador'] = 'Adam'

model

### Ajuste

In [ ]:
num_epochs = 15
early_stopping_patience = 5

exp_config_DenseNet_121Modificado['num_epochs'] = num_epochs
exp_config_DenseNet_121Modificado['early_stopping_patience'] = early_stopping_patience

checkpoint_path = './best_model_Inception.pth'

train_and_validate(model, train_dataloader, val_dataloader, criterion, optimizer, device, num_epochs, early_stopping_patience, checkpoint_path)

### Test

In [ ]:
model = DenseNet_121CNN().to(device)

model.load_state_dict(torch.load(checkpoint_path))
model.to(device)

model.eval()

In [ ]:
y_true, y_proba = test(model, test_dataloader, device)

In [ ]:
y_true, y_pred, y_proba_flat = classify(y_proba, y_true)

In [ ]:
accuracy, precision, recall, specificity, conf_matrix, fpr, tpr, roc_auc = calculate_metrics(y_true, y_pred, y_proba_flat)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
exp_config_DenseNet_121Modificado

In [ ]:
wandb.finish()